## サンプルサイト

http://crawler.sbcr.jp/samplepage.html

## HTML解析

* エンコーディングの統一
    * UTF-8以外の場合はエンコードをそろえる
* 情報の確認
    * HTMLパーサーを使ってXPath指定で取り出す
    * それくらいで無ければ正規表現で取り出す

## 日付と記事リンクの取り出し

```
<div class="topicsCenterColumnTopicsListDate">
     2014年2月12日<br />
</div>
```

In [1]:
page_source = open("samplepage.html", &:read)
dates = page_source.scan(%r!(\d+)年 ?(\d+)月  ?(\d+)日<br />!)
dates[0,4]

[["2014", "2", "6"], ["2014", "1", "9"], ["2014", "1", "9"], ["2013", "12", "5"]]

In [2]:
url_titles = page_source.scan(%r!^<a href="(.+?)">(.+?)</a><br />!)
puts url_titles[0,4]

http://www.sbcr.jp/topics/11719/
最強の布陣で挑む！ GA文庫電子版【俺TUEEEEE】キャンペーン開催中
http://www.sbcr.jp/topics/11712/
【新刊情報】2014年2月17日～23日　「コンセプト」の作り方がわかるビジネス書など12点
http://www.sbcr.jp/topics/11710/
『数学ガール』電子書籍版がAmazon Kindleで配信開始！ キャンペーンも同時開催！！
http://www.sbcr.jp/topics/11703/
【新刊情報】2014年2月10日～16日　アニメ化決定『ワルブレ』最新刊など11点


In [3]:
## 日付とタイトルの数が一致しているか確認
p dates.length
p url_titles.length

16
68


68

In [4]:
## Array#zipで取得
p dates[0,2]
p dates[0,2].zip(url_titles[0,2])

[["2014", "2", "6"], ["2014", "1", "9"]]
[[["2014", "2", "6"], ["http://www.sbcr.jp/topics/11719/", "最強の布陣で挑む！ GA文庫電子版【俺TUEEEEE】キャンペーン開催中"]], [["2014", "1", "9"], ["http://www.sbcr.jp/topics/11712/", "【新刊情報】2014年2月17日～23日　「コンセプト」の作り方がわかるビジネス書など12点"]]]


[[["2014", "2", "6"], ["http://www.sbcr.jp/topics/11719/", "最強の布陣で挑む！ GA文庫電子版【俺TUEEEEE】キャンペーン開催中"]], [["2014", "1", "9"], ["http://www.sbcr.jp/topics/11712/", "【新刊情報】2014年2月17日～23日　「コンセプト」の作り方がわかるビジネス書など12点"]]]

In [5]:
## 日本時間のTimeオブジェクト作成
puts Time.local "2014", "2", "20"
puts Time.local 2014, 2, 20
puts Time.local dates[0][0], dates[0][1], dates[0][2]

2014-02-20 00:00:00 +0900
2014-02-20 00:00:00 +0900
2014-02-06 00:00:00 +0900


## HTMLアンエスケープ

* HTMLは特定の記号に独自の記法が定めてある
    * 正規表現で抜き出しただけだとこれらが残るのでアンエスケープしておく
        * CGI.unescapeHTMLを利用
```
require 'cgi'
CGI.unescapeHTML "&lt;A&amp;B&gt;"  ### これは「<A&B>」という表示になる

```

In [10]:
require 'cgi'

def parse(page_source)
  dates = page_source.scan(%r!(\d+)年 ?(\d+)月  ?(\d+)日<br />!)
  url_titles = page_source.scan(%r!^<a href="(.+?)">(.+?)</a><br />!)
#   url_titles.zip(dates).map{
#     |(aurl, atitle), ymd|
#     [ CGI.unescapeHTML(aurl), 
#       CGI.unescapeHTML(atitle), 
#       Time.local(*ymd)
#     ] 
#   }
  url_titles.zip(dates)
end
  
x = parse(open("samplepage.html", &:read))
x[0,2][0]

[["http://www.sbcr.jp/topics/11719/", "最強の布陣で挑む！ GA文庫電子版【俺TUEEEEE】キャンペーン開催中"], ["2014", "2", "6"]]

## WgetをRubyから呼ぶ

In [15]:
x = parse(`/usr/bin/wget -q -O- http://crawler.sbcr.jp/samplepage.html`)
x[0,2][0]

[["http://www.sbcr.jp/topics/11719/", "最強の布陣で挑む！ GA文庫電子版【俺TUEEEEE】キャンペーン開催中"], ["2014", "2", "6"]]

In [3]:
require 'date'
require 'nyaplot'
require 'yahoo-finance'

#Yahoo Financeから金価格を取得する
gold_prices = YahooFinance.historical_quotes(
  "GC=F", #金のシンボル
  Date.today.prev_month, #30日前から
  Date.today
).map do |row|
  { date: Date.parse(row.date), price: row.close.to_f }
end

#Yahoo Financeからビットコイン価格を取得する
btc_prices = YahooFinance.historical_quotes(
  "BTC-USD", #ビットコインのシンボル
  Date.today.prev_month, #30日前から
  Date.today
).map do |row|
  { date: Date.parse(row.date), price: row.close.to_f }
end


#Nyaplotで2つのチャートを描画する
plot = Nyaplot::Plot.new
plot.x_label("Date")
plot.y_label("Price")

gold_scatter = plot.add_with_df(Nyaplot::DataFrame.new(gold_prices))
gold_scatter.x({ key: :date })
gold_scatter.y({ key: :price })
gold_scatter.title("Gold Price")

btc_scatter = plot.add_with_df(Nyaplot::DataFrame.new(btc_prices))
btc_scatter.x({ key: :date })
btc_scatter.y({ key: :price })
btc_scatter.title("Bitcoin Price")

#plot.show

NoMethodError: undefined method `historical_quotes' for YahooFinance:Module

In [1]:
require 'date'
require 'nyaplot'
require 'yahoo-finance'

#Yahoo Financeから金価格を取得する
gold_prices = YahooFinance::Client.new.historical_quotes(
  "GC=F", #金のシンボル
  start_date: Date.today.prev_month, #30日前から
  end_date: Date.today
).map do |row|
  { date: Date.parse(row.trade_date), price: row.close.to_f }
end

#Yahoo Financeからビットコイン価格を取得する
btc_prices = YahooFinance::Client.new.historical_quotes(
  "BTC-USD", #ビットコインのシンボル
  start_date: Date.today.prev_month, #30日前から
  end_date: Date.today
).map do |row|
  { date: Date.parse(row.trade_date), price: row.close.to_f }
end

#Nyaplotで2つのチャートを描画する
plot = Nyaplot::Plot.new
plot.x_label("Date")
plot.y_label("Price")

gold_scatter = plot.add_with_df(Nyaplot::DataFrame.new(gold_prices))
gold_scatter.x({ key: :date })
gold_scatter.y({ key: :price })
gold_scatter.title("Gold Price")

btc_scatter = plot.add_with_df(Nyaplot::DataFrame.new(btc_prices))
btc_scatter.x({ key: :date })
btc_scatter.y({ key: :price })
btc_scatter.title("Bitcoin Price")

plot.show

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

Errno::ENOENT: No such file or directory @ rb_sysopen - https://ichart.finance.yahoo.com/table.csv?s=GC=F&g=d&ignore=.csv&a=1&b=19&c=2023&d=2&e=19&f=2023

In [2]:
fuga = YahooFinance::Client.new
fuga.historical_quotes("BTC-USD")

Errno::ENOENT: No such file or directory @ rb_sysopen - https://ichart.finance.yahoo.com/table.csv?s=BTC-USD&g=d&ignore=.csv